In [1]:
import os

In [2]:
%pwd

'c:\\Users\\ankita\\Desktop\\Personal_Project\\DL_Project\\chicken_disease_classification\\research'

In [3]:
os.chdir("../../")

In [4]:
import tensorflow as tf

In [7]:
model = tf.keras.models.load_model("artifacts/training/model.keras")

In [8]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class EvaluationConfig:
    path_of_model:Path
    training_data:Path
    all_params: dict
    params_image_size:list
    params_batch_size:int

In [9]:
from ChickenDiseaseClassification.constants import *
from ChickenDiseaseClassification.utils.common import read_yaml, create_directories, save_json

In [20]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH
        ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifact_root])
        
    def get_validation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.keras",
            training_data= "artifacts/data_ingestion/poultry_diseases",
            all_params = self.params,
            params_image_size= self.params.IMAGE_SIZE,
            params_batch_size= self.params.BATCH_SIZE
        )
        
        return eval_config


In [21]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config
        
    def valid_generator(self):
        detagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split = 0.30
        )
        
        dataflow_kwargs = dict(
            target_size = self.config.params_image_size[:-1],
            batch_size = self.config.params_batch_size,
            interpolation = "bilinear"
        )
        
        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **detagenerator_kwargs
        )
        
        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory = self.config.training_data,
            subset = "validation",
            shuffle = False,
            **dataflow_kwargs
        )
        
    @staticmethod
    def load_model(path:Path)-> tf.keras.Model:
        return tf.keras.models.load_model(path)
    
    def evalution(self):
        self.model = self.load_model(self.config.path_of_model)
        self.valid_generator()
        self.score = model.evaluate(self.valid_generator)
        
    def save_score(self):
        scores = {'loss': self.score[0], "accuracy": self.score[1]}
        save_json(path = Path("scored.json"), data = scores)

In [22]:
try:
    config = ConfigurationManager()
    val_config = config.get_validation_config()
    evaluation = Evaluation(val_config)
    evaluation.evalution()
    evaluation.save_score()
    
except Exception as e:
    raise e
    

[2024-09-07 20:49:26,900: INFO : common: yaml file: chicken_disease_classification\config\config.yaml loaded successfully]
[2024-09-07 20:49:26,902: INFO : common: yaml file: chicken_disease_classification\params.yaml loaded successfully]
[2024-09-07 20:49:26,903: INFO : common: created directory at: aritfacts]
Found 1247 images belonging to 2 classes.


C:\Users\ankita\Desktop\Personal_Project\DL_Project\venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


78/78 ━━━━━━━━━━━━━━━━━━━━ 163s 2s/step - accuracy: 0.9227 - loss: 0.4176
[2024-09-07 20:52:11,539: INFO : common: json file saved at: scored.json]
